# 3) Exclamation Points: Rules vs. Reality

**Goal:** Count exclamation points per 10k words and compare.

# Setup: Load Texts

This notebook needs **Alice in Wonderland** and **Through the Looking-Glass** as input texts.

**How to provide the texts:**
1. Download books from Project Gutenberg (IDs 11 and 12) as txts. [go to https://www.gutenberg.org/ebooks/11 and https://www.gutenberg.org/ebooks/12]

2. Place two text files in the "data" folder with names:
   - `Wondeland.txt`  (Alice's Adventures in Wonderland)
   - `Looking-Glass.txt` (Through the Looking-Glass)

In [ ]:
import re
from pathlib import Path

In [ ]:

def load_texts(local_alice: str = '../data/Wonderland.txt',
               local_glass: str = '../data/Looking-Glass.txt'):
    """Load Wonderland and Looking-Glass texts from disk.

    Parameters
    ----------
    local_alice : str
        Path to Wonderland text file. Defaults to '../data/Wonderland.txt'.
    local_glass : str
        Path to Looking-Glass text file. Defaults to '../data/Looking-Glass.txt'.

    Returns
    -------
    tuple[str, str]
        (wonderland_text, lookingglass_text).

    Raises
    ------
    FileNotFoundError
        If either file is missing.

    Extra Notes
    -----------
    - Using UTF-8 with `errors='ignore'` avoids codec exceptions on
      older Project Gutenberg dumps or inconsistent encodings.
    """
    p1, p2 = Path(local_alice), Path(local_glass)

    # Fail fast with a clear message if a file is missing
    if not p1.exists():
        raise FileNotFoundError(
            f"Missing file: {p1}\n"
            "→ Please place 'Wonderland.txt' at this path or update load_texts(...)."
        )
    if not p2.exists():
        raise FileNotFoundError(
            f"Missing file: {p2}\n"
            "→ Please place 'Looking-Glass.txt' at this path or update load_texts(...)."
        )

    # Read the files (UTF-8; ignore undecodable bytes to stay robust)
    wonderland   = p1.read_text(encoding='utf-8', errors='ignore')
    lookingglass = p2.read_text(encoding='utf-8', errors='ignore')
    return wonderland, lookingglass

def normalize(text: str) -> str:
    """Normalize a Gutenberg-like text for tokenization.

    Steps
    -----
    1) Heuristically strip Project Gutenberg headers/footers if present
       (looks for *** START ... *** END markers).
    2) Normalize newlines to '\n'.

    Parameters
    ----------
    text : str
        Raw text as loaded from disk (can be empty).

    Returns
    -------
    str
        Cleaned text suitable for tokenization and counting.
    """
    if not text:
        return ''
    # Clip to the main body if markers are present.
    start = text.find('*** START')
    end   = text.find('*** END')
    if start != -1 and end != -1 and end > start:
        text = text[start:end]
    # Normalize Windows line endings.
    return text.replace('\r\n', '\n')

# Load raw texts (forgiving: returns '' if a file is missing)
wonderland_raw, lookingglass_raw = load_texts()

# Normalize for tokenization
wonderland   = normalize(wonderland_raw)
lookingglass = normalize(lookingglass_raw)

print(f"Wonderland chars: {len(wonderland):,} | Looking-Glass chars: {len(lookingglass):,}")


In [ ]:
WORD_RE = re.compile(r"[A-Za-z']+")  # keep apostrophes in words (e.g., don't -> don't)

def words(text: str):
    """Simple word tokenizer (lowercased, ASCII letters + apostrophes).

    Pros
    ----
    - Very fast and dependency-free.
    - Good enough for frequency/keyness demonstrations.

    Cons
    ----
    - No punctuation words, no sentence boundaries, no POS tags.
    - May treat possessives inconsistently across sources.

    Returns
    -------
    list[str]
        Lowercased word words.
    """
    return WORD_RE.findall(text.lower())


def sentences(text: str):
    """Naive sentence splitter using punctuation boundaries.

    Uses a regex to split on '.', '!', '?' followed by whitespace.
    Because this is heuristic, treat results as approximate.

    Returns
    -------
    list[str]
        Sentence-like strings.
    """
    return [s.strip() for s in re.split(r'(?<=[.!?])\s+', text) if s.strip()]



wonderland_words = words(wonderland)
lookingglass_words = words(lookingglass)

wonderland_sentences = sentences(wonderland)
lookingglass_sentences = sentences(lookingglass)

print(f"Wonderland words: {len(wonderland_words):,} | Looking-Glass words: {len(lookingglass_words):,}")
print(f"Wonderland sentences: {len(wonderland_sentences):,} | Looking Glass sentences: {len(lookingglass_sentences):,}")


### Count and Normalize

In [ ]:

def exclamations_per_10k(text, tokens):
    count = text.count('!')
    per_10k = (count / max(1,len(tokens))) * 10000
    return count, per_10k

a_count, a_rate = exclamations_per_10k(wonderland, words(wonderland))
g_count, g_rate = exclamations_per_10k(lookingglass, words(lookingglass))
print(f"Wonderland: {a_count} total | {a_rate:.2f} per 10k words")
print(f"Looking Glass: {g_count} total | {g_rate:.2f} per 10k words")


**Question:** Sample passages with many exclamation points. How do they shape voice, pacing, or mood?

In [ ]:
# A) Sentence-level hotspots
def top_exclaim_sentences(sents, top_n=8, min_len=20):
    scored = [(s.count('!'), len(s), s) for s in sents if len(s) >= min_len]
    scored.sort(key=lambda x: (x[0], -x[1]), reverse=True)  # more !, then longer
    return [(cnt, s) for cnt, _, s in scored[:top_n] if cnt > 0]

# B) Clusters over sliding windows (tempo spikes)
def exclaim_clusters(sents, window=6, min_total=3, top_k=5):
    out = []
    for i in range(max(0, len(sents)-window+1)):
        chunk = " ".join(sents[i:i+window])
        c = chunk.count('!')
        if c >= min_total:
            out.append((c, i, " ".join(sents[i:i+window])))
    out.sort(reverse=True, key=lambda x: x[0])
    return out[:top_k]

def preview(s, n=300):
    return s if len(s) <= n else s[:n].rstrip() + " …"


print("=== Wonderland: top sentences with ! ===")
for cnt, s in top_exclaim_sentences(wonderland_sentences):
    print(f"[! x{cnt}] {preview(s)}\n")

print("=== Looking-Glass: top sentences with ! ===")
for cnt, s in top_exclaim_sentences(lookingglass_sentences):
    print(f"[! x{cnt}] {preview(s)}\n")

print("=== Wonderland: exclamation clusters ===")
for c, i, chunk in exclaim_clusters(wonderland_sentences, window=6, min_total=3):
    print(f"[cluster ! x{c} | sentences {i}-{i+5}] {preview(chunk)}\n")

print("=== Looking-Glass: exclamation clusters ===")
for c, i, chunk in exclaim_clusters(lookingglass_sentences, window=6, min_total=3):
    print(f"[cluster ! x{c} | sentences {i}-{i+5}] {preview(chunk)}\n")
